In [1]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetPt, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf
# import sys
# sys.path.append("/home/kvake/.local/lib/python3.11/site-packages")
import optuna
# sys.path.append("/home/kvake/.local/lib/python3.9/site-packages")  # Adjust if needed
import optuna_dashboard

2025-04-04 01:13:35.630448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743722015.643240  521996 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743722015.647148  521996 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 01:13:35.662835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Available GPUs: []


2025-04-04 01:13:39.843138: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"

In [ ]:
dataset = Dataset()
# dataset.build_dataset()
# dataset.save_data()
dataset.load_data()

print(len(dataset.train_dataset))

#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
# data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

# plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
# plt.legend(loc='upper right')
# plt.title('pt distribution of the dataset')
# plt.xlabel('pt')
# plt.ylabel('Number of events')
# plt.show()

6523684


In [4]:
import optuna
from optuna_dashboard import run_server
from DatasetClass import DatasetPt
from ModelClass import RegressionModel
import threading

# ======================
# 1. Build/Load Dataset (Outside Optuna Loop for Efficiency)
# ======================
dataset = DatasetPt()
dataset.load_data()

print("Train dataset size:", len(dataset.train_dataset))

# ============================
# 2. Define Optuna Objective (Fast Training)
# ============================
def objective(trial):
    """
    Optuna objective function for hyperparameter search.
    """
    model = RegressionModel(
        dataset,
        n_layers=trial.suggest_int('n_layers', 3, 6),
        hidden_layer_size=2048,
        initial_learning_rate=trial.suggest_float('initial_learning_rate', 1e-3, 1e-2, log=True),
        n_epochs=80,
        activation_function  = 'relu',
        batch_size=trial.suggest_int('batch_size', 3000, 5000, step=1000),
        dropout_rate=0.2,
        weight_decay= 1e-5,
        optimizer='adamw',
        n_normalizer_samples = trial.suggest_int('n_normalizer_samples', 100, 10000, step=1000)
    )

    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()

    val_loss, _ = model.model.evaluate(model.val_batch, verbose=0)
    return val_loss

# ========================
# 3. Run the Optuna Study
# ========================
study = optuna.create_study(storage="sqlite:///optuna.db", study_name="Higgs_analysis_jupyter_5", direction='minimize')

# ========================
# 4. Start `optuna-dashboard` in a Separate Thread
# ========================

def run_dashboard():
    server = run_server("sqlite:///optuna.db", host="0.0.0.0", port=8080)
    server.run()  # Run dashboard continuously in a separate thread

dashboard_thread = threading.Thread(target=run_dashboard)
dashboard_thread.daemon = True  # Ensures it stops when the script ends
dashboard_thread.start()

def run_optuna():
    study.optimize(objective, n_trials=10, n_jobs=1)

# Run Optuna in a separate thread so the dashboard can be accessed in real-time
optuna_thread = threading.Thread(target=run_optuna)
optuna_thread.start()

# ========================
# 5. Print Results
# ========================
optuna_thread.join()  # Wait for Optuna to finish before printing results
print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)

# Dashboard will keep running in background


Train dataset size: 6523684


[I 2025-04-04 01:13:56,351] A new study created in RDB with name: Higgs_analysis_jupyter_5
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/optuna/distributions.py:708: UserWarning: The distribution is specified by [100, 10000] and step=1000, but the range is not divisible by `step`. It will be replaced by [100, 9100].
  warnings.warn(


Batching datasets...
Building model...
Epoch 1/80


2025-04-04 01:13:56.930012: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


 112/1631 ━━━━━━━━━━━━━━━━━━━━ 4:52 192ms/step - loss: 23281.1738 - mean_absolute_percentage_error: 88.8686 - mean_squared_error: 23281.1719

KeyboardInterrupt: 